Danish Deepak

## Project 19: Detect Credit Card Fraud

In [1]:
# Importing the necessary libraries

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression 
from sklearn.metrics import r2_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import warnings 
warnings.filterwarnings('ignore')

In [2]:
# Importing the dataset
df = pd.read_csv('creditcard.csv')

### 1) Data Exploration

In [3]:
# Looking at the dataset
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# Learning the dtypes of each column and also checking for null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
# There are no null values
# All the columns are of float dtype except the column class which of int dtype

In [6]:
# Basic statistical analysis
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.893285e-16,-3.147640e-15,...,1.473120e-16,8.042109e-16,5.282512e-16,4.456271e-15,1.426896e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [7]:
# Lets check the class column
print(len(df['Class'].unique()))
print(df['Class'].unique())

2
[0 1]


In [8]:
# The class column has only 2 unique values so that makes it our target feature 

### 2) Data transformation

In [9]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop('Class', axis = 1))
df_scaled = pd.DataFrame(df_scaled, columns = [df.drop('Class', axis = 1).columns])
df_scaled.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.996583,-0.694242,-0.044075,1.672773,0.973366,-0.245117,0.347068,0.193679,0.082637,0.331128,...,0.326118,-0.024923,0.382854,-0.176911,0.110507,0.246585,-0.392170,0.330892,-0.063781,0.244964
1,-1.996583,0.608496,0.161176,0.109797,0.316523,0.043483,-0.061820,-0.063700,0.071253,-0.232494,...,-0.089611,-0.307377,-0.880077,0.162201,-0.561131,0.320694,0.261069,-0.022256,0.044608,-0.342475
2,-1.996562,-0.693500,-0.811578,1.169468,0.268231,-0.364572,1.351454,0.639776,0.207373,-1.378675,...,0.680975,0.337632,1.063358,1.456320,-1.138092,-0.628537,-0.288447,-0.137137,-0.181021,1.160686
3,-1.996562,-0.493325,-0.112169,1.182516,-0.609727,-0.007469,0.936150,0.192071,0.316018,-1.262503,...,-0.269855,-0.147443,0.007267,-0.304777,-1.941027,1.241904,-0.460217,0.155396,0.186189,0.140534
4,-1.996541,-0.591330,0.531541,1.021412,0.284655,-0.295015,0.071999,0.479302,-0.226510,0.744326,...,0.529939,-0.012839,1.100011,-0.220123,0.233250,-0.395202,1.041611,0.543620,0.651816,-0.073403


In [10]:
# Splitting the data into train test sets

x = df_scaled
y = df['Class']

x_train, x_test, y_train, y_test  = train_test_split(x, y, test_size = 0.3, random_state = 7)

### 3) Data Modelling using Linear Regression

In [11]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
score  = r2_score(y_test, y_pred)
print(score)

0.5787266111487874


### 4) Data modelling using multiple ML Algorithms

In [12]:
# Lasso regression
al = [0.1, 0.01, 0.0001]
for i in al:
    lasso  = Lasso(alpha = i)
    lasso.fit(x_train, y_train)
    y_pred = lasso.predict(x_test)
    score  = r2_score(y_test, y_pred)
    print(f'For alpha {i} r2_score: {score}')

For alpha 0.1 r2_score: -8.450241923529944e-06
For alpha 0.01 r2_score: 0.07053947754774881
For alpha 0.0001 r2_score: 0.5784462169211451


In [13]:
# Ridge Regression
al = [0.1, 1, 10, 100]
for i in al:
    ridge  = Ridge(alpha = i)
    ridge.fit(x_train, y_train)
    y_pred = ridge.predict(x_test)
    score  = r2_score(y_test, y_pred)
    print(f'For alpha {i} r2_score: {score}')

For alpha 0.1 r2_score: 0.5787266079482224
For alpha 1 r2_score: 0.5787265791237
For alpha 10 r2_score: 0.5787262889551926
For alpha 100 r2_score: 0.5787231960136539


In [14]:
# Logistic regression
logreg  = LogisticRegression()
logreg.fit(x_train,y_train)
y_pred  = logreg.predict(x_test)
score   = accuracy_score(y_pred, y_test)
con_mat = confusion_matrix(y_pred, y_test)
print('Confusion_matrix:\n ',con_mat )
print('Accuracy: ',score)

Confusion_matrix:
  [[85276    63]
 [   12    92]]
Accuracy:  0.9991222218320986


In [15]:
# Random Forest Classifier
rf      = RandomForestClassifier()
rf.fit(x_train, y_train)
y_pred  = rf.predict(x_test)
con_mat = confusion_matrix(y_pred, y_test)
score   = accuracy_score(y_pred, y_test)
print('Confusion Matrix:\n', con_mat,'\nAccuracy Score: ', score)

Confusion Matrix:
 [[85280    26]
 [    8   129]] 
Accuracy Score:  0.999602073897218


In [16]:
# Support Vector Machines
svm     = SVC()
svm.fit(x_train, y_train)
y_pred  = svm.predict(x_test)
con_mat = confusion_matrix(y_pred, y_test)
score   = accuracy_score(y_pred, y_test)
print('Confusion Matrix:\n', con_mat,'\nAccuracy Score: ', score)

Confusion Matrix:
 [[85281    48]
 [    7   107]] 
Accuracy Score:  0.9993562960102056


In [17]:
# Gradient Boosting classifier
gb      = GradientBoostingClassifier()
gb.fit(x_train,y_train)
y_pred  = gb.predict(x_test)
con_mat = confusion_matrix(y_pred, y_test)
score   = accuracy_score(y_pred, y_test)
print('Confusion Matrix:\n', con_mat,'\nAccuracy Score: ', score)

Confusion Matrix:
 [[85257    56]
 [   31    99]] 
Accuracy Score:  0.9989817773252344


In [18]:
# Decision tree classifier
dt      = DecisionTreeClassifier()
dt.fit(x_test, y_test)
y_pred  = dt.predict(x_test)
con_mat = confusion_matrix(y_pred, y_test)
score   = accuracy_score(y_pred, y_test)
print('Confusion Matrix:\n', con_mat,'\nAccuracy Score: ', score)

Confusion Matrix:
 [[85288     0]
 [    0   155]] 
Accuracy Score:  1.0


The classification ML algorithms gives the best accuracy as we see above